In [1]:
import sqlite3
import random
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Создание базы данных
connection = sqlite3.connect('my_database.db')
cursor = connection.cursor()

# Создание таблиц
cursor.execute('''
CREATE TABLE IF NOT EXISTS Users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    registration_date DATE NOT NULL,
    email TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL,
    quantity INTEGER NOT NULL,
    created_at DATE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES Users (id),
    FOREIGN KEY (product_id) REFERENCES Products (id)
)
''')

connection.commit()


In [3]:
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

In [4]:
# Заполнение таблицы Users
for _ in range(10):
    name = f'User{random.randint(1, 100)}'
    age = random.randint(18, 65)
    registration_date = random_date(datetime(2020, 1, 1), datetime.now()).date()
    email = f'{name.lower()}@example.com'
    cursor.execute("INSERT INTO Users (name, age, registration_date, email) VALUES (?, ?, ?, ?)",
                   (name, age, registration_date, email))

# Заполнение таблицы Products
for _ in range(10):
    name = f'Product{random.randint(1, 100)}'
    price = round(random.uniform(10.0, 500.0), 2)
    quantity = random.randint(1, 100)
    created_at = datetime.now().date()
    cursor.execute("INSERT INTO Products (name, price, quantity, created_at) VALUES (?, ?, ?, ?)",
                   (name, price, quantity, created_at))

# Заполнение таблицы Orders
for _ in range(20):
    user_id = random.randint(1, 10)
    product_id = random.randint(1, 10)
    order_date = random_date(datetime(2020, 1, 1), datetime.now()).date()
    quantity = random.randint(1, 5)
    cursor.execute("INSERT INTO Orders (user_id, product_id, order_date, quantity) VALUES (?, ?, ?, ?)",
                   (user_id, product_id, order_date, quantity))

connection.commit()

C:\Users\bulga\AppData\Local\Temp\ipykernel_14572\1595377038.py:7: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("INSERT INTO Users (name, age, registration_date, email) VALUES (?, ?, ?, ?)",
C:\Users\bulga\AppData\Local\Temp\ipykernel_14572\1595377038.py:16: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("INSERT INTO Products (name, price, quantity, created_at) VALUES (?, ?, ?, ?)",
C:\Users\bulga\AppData\Local\Temp\ipykernel_14572\1595377038.py:25: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("INSERT INTO Orders (user_id, product_id, order_date, quantity) VALUES (?, ?, ?, ?)",


In [5]:
# Выбираем имена и возраст пользователей старше 25 лет
cursor.execute('''
SELECT o.id, o.user_id, u.name, u.email, o.product_id, p.name, o.order_date, o.quantity
FROM Orders o LEFT JOIN Users u ON o.user_id = u.id
			        LEFT JOIN Products p ON o.product_id = p.id
''')
results = cursor.fetchall()

connection.close()

In [6]:
# Создадим датафрейм
df = pd.DataFrame(results, columns=['order_id', 'user_id', 'user_name', \
                                    'user_email', 'product_id', 'product_name', \
                                    'order_date', 'order_quantity'])

In [7]:
df

,order_id,user_id,user_name,user_email,product_id,product_name,order_date,order_quantity
0,1,7,User34,user34@example.com,4,Product64,2022-11-09,5
1,2,6,User57,user57@example.com,9,Product12,2022-10-11,2
2,3,9,User14,user14@example.com,3,Product49,2020-01-14,3
3,4,3,User6,user6@example.com,9,Product12,2021-01-01,5
4,5,1,User82,user82@example.com,6,Product50,2021-10-09,3
5,6,9,User14,user14@example.com,10,Product47,2024-08-14,2
6,7,9,User14,user14@example.com,4,Product64,2025-04-06,5
7,8,8,User90,user90@example.com,6,Product50,2023-06-23,4
8,9,9,User14,user14@example.com,6,Product50,2021-02-25,4
9,10,8,User90,user90@example.com,1,Product21,2021-03-31,5


In [8]:
# Сохраним нашу базу в excel
df.to_excel('our_base.xlsx')